<a href="https://colab.research.google.com/github/kurehajime/openapi_sandbox/blob/main/function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00


In [3]:
import openai
openai.api_key = secret

In [39]:
def completion(prompt):
  response =  openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": prompt}],
    functions=[
        {
            "name": "birthday_horoscope",
            "description": "誕生日で今日の運勢を占えるよ",
            "parameters": {
                "type": "object",
                "properties": {
                    "year": {
                        "type": "integer",
                        "description": "年",
                    },
                    "month": {
                        "type": "integer",
                        "description": "月(1はじまり)",
                    },
                    "day": {
                        "type": "integer",
                        "description": "日",
                    },
                },
                "required": ["year","month","day"],
            },
        }
    ],
    function_call="auto",
  )
  return response["choices"][0]["message"]

In [43]:
print(completion("サンフランシスコ講和条約を結んだ日本の総理大臣の運勢を教えて"))

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "birthday_horoscope",
    "arguments": "{\n  \"year\": 1951,\n  \"month\": 9,\n  \"day\": 8\n}"
  }
}


In [42]:
print(completion("Nintendo64がPlayStationに負けた理由を英語で教えて"))

{
  "role": "assistant",
  "content": "There were several reasons why the Nintendo 64 lost to the PlayStation:\n\n1. CD-ROM Format: The PlayStation used CDs as its storage medium, while the N64 used cartridges. CDs had larger storage capacity and were cheaper to produce, allowing for more complex and longer games on the PlayStation.\n\n2. Third-Party Support: Many game developers chose to support the PlayStation platform due to its CD format and larger market share. This led to a wider library of games available on the PlayStation, while the N64 had a more limited selection.\n\n3. Cost: The N64 was more expensive to produce than the PlayStation. The higher price point made it less accessible to a wider audience, especially when compared to the more affordable PlayStation.\n\n4. Marketing: Sony's aggressive marketing strategy helped establish the PlayStation brand as a household name. They targeted a broader demographic, including older gamers and non-traditional gaming audiences, which

In [67]:
import json
def uranai(message):
  if message["function_call"]["name"] == "birthday_horoscope":
    arguments = json.loads(message["function_call"]["arguments"])
    year = arguments["year"]
    month = arguments["month"]
    day = arguments["day"]
    num = (year + month + day) % 3
    if num == 0 :
      return "大吉"
    if num == 1:
      return "吉"
    if num == 2:
      return "凶"

In [70]:
message = completion("サンフランシスコ講和条約を結んだ日本の総理大臣の運勢を教えて")
if message.get("function_call"):
  print(uranai(message))
if message.get("function_call"):
  print(message["content"])

大吉
None


In [71]:
def completion2(prompt):
  message = completion(prompt)
  if message.get("function_call"):
    kekka = uranai(message)
    response =  openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0613",
      messages=[
          {"role": "user", "content": prompt},
          message,
          {"role": "function", "name":"birthday_horoscope", "content": kekka},
          ],
    )
    return response["choices"][0]["message"]["content"]

In [74]:
print(completion2("サンフランシスコ講和条約を結んだ日本の総理大臣の運勢をハイテンションで教えて❤❤❤"))

サンフランシスコ講和条約を結んだ日本の総理大臣は、運勢が大吉となります！ハイテンションで、素晴らしい未来が待っています❤❤❤成功への道が開け、幸運な出来事や好機が訪れるでしょう。周囲の人々からも信頼と尊敬を集め、絶大な支持を得ることができます。また、リーダーシップと決断力を発揮することで、大きな成果を上げることができますよ。自信を持って前に進み、困難を乗り越えることができるでしょう。さらに、幸福な家庭生活や健康な身体も享受できるはずです。この期間は、大きな目標を追い求め、夢を実現するチャンスとなりますので、積極的に行動してください！
